In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1974339,2021-12-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1974340,2021-12-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1974341,2021-12-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1974342,2021-12-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
51107,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
51109,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
51111,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
51113,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
51115,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1199185,2021-12-26,Arkansas,Arkansas,5001,3604,64.00,5000,Arkansas,AR,Arkansas,State,3017804
1199187,2021-12-27,Arkansas,Arkansas,5001,3606,64.00,5000,Arkansas,AR,Arkansas,State,3017804
1199189,2021-12-28,Arkansas,Arkansas,5001,3612,64.00,5000,Arkansas,AR,Arkansas,State,3017804
1199191,2021-12-29,Arkansas,Arkansas,5001,3638,64.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1974339,2021-12-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1974340,2021-12-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1974341,2021-12-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1974342,2021-12-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-12-30') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
709,2021-12-30,Snohomish,Washington,80290,841.00,53061,WA,County,822083,9766.65,102.30
1416,2021-12-30,Cook,Illinois,821472,12541.00,17031,IL,County,5150233,15950.19,243.50
2122,2021-12-30,Orange,California,357716,5890.00,6059,CA,County,3175692,11264.19,185.47
2827,2021-12-30,Maricopa,Arizona,866014,13670.00,4013,AZ,County,4485414,19307.34,304.77
3532,2021-12-30,Los Angeles,California,1670249,27625.00,6037,CA,County,10039107,16637.43,275.17
...,...,...,...,...,...,...,...,...,...,...,...
1972691,2021-12-30,Wheeler,Oregon,139,2.00,41069,OR,County,1332,10435.44,150.15
1973135,2021-12-30,King,Texas,27,0.00,48269,TX,County,272,9926.47,0.00
1973548,2021-12-30,Esmeralda,Nevada,63,2.00,32009,NV,County,873,7216.49,229.10
1973957,2021-12-30,Loving,Texas,22,0.00,48301,TX,County,169,13017.75,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
709,2021-12-30,Snohomish,Washington,80290,841.00,53061,WA,County,822083,9766.65,102.30,102.30,9766.65
1416,2021-12-30,Cook,Illinois,821472,12541.00,17031,IL,County,5150233,15950.19,243.50,243.50,15950.19
2122,2021-12-30,Orange,California,357716,5890.00,6059,CA,County,3175692,11264.19,185.47,185.47,11264.19
2827,2021-12-30,Maricopa,Arizona,866014,13670.00,4013,AZ,County,4485414,19307.34,304.77,304.77,19307.34
3532,2021-12-30,Los Angeles,California,1670249,27625.00,6037,CA,County,10039107,16637.43,275.17,275.17,16637.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972691,2021-12-30,Wheeler,Oregon,139,2.00,41069,OR,County,1332,10435.44,150.15,150.15,10435.44
1973135,2021-12-30,King,Texas,27,0.00,48269,TX,County,272,9926.47,0.00,0.00,9926.47
1973548,2021-12-30,Esmeralda,Nevada,63,2.00,32009,NV,County,873,7216.49,229.10,229.10,7216.49
1973957,2021-12-30,Loving,Texas,22,0.00,48301,TX,County,169,13017.75,0.00,0.00,13017.75


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1254402,2021-12-30,Hale,Alabama,3295,91.00,1065,AL,County,14651,22489.93,621.12,621.12,22489.93,1
1253761,2021-12-30,Fayette,Alabama,3407,86.00,1057,AL,County,16302,20899.28,527.54,527.54,20899.28,2
727356,2021-12-30,Franklin,Alabama,6548,111.00,1059,AL,County,31362,20878.77,353.93,353.93,20878.77,3
1366497,2021-12-30,Clarke,Alabama,4930,88.00,1025,AL,County,23622,20870.38,372.53,372.53,20870.38,4
1058266,2021-12-30,Winston,Alabama,4926,111.00,1133,AL,County,23629,20847.26,469.76,469.76,20847.26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196624,2021-12-30,Converse,Wyoming,2386,48.00,56009,WY,County,13822,17262.34,347.27,347.27,17262.34,19
1055046,2021-12-30,Johnson,Wyoming,1389,17.00,56019,WY,County,8445,16447.60,201.30,201.30,16447.60,20
1490909,2021-12-30,Lincoln,Wyoming,3087,27.00,56023,WY,County,19830,15567.32,136.16,136.16,15567.32,21
1197267,2021-12-30,Sublette,Wyoming,1527,23.00,56035,WY,County,9831,15532.50,233.95,233.95,15532.50,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
979581,2021-12-30,Lowndes,Alabama,1855,68.00,1085,AL,County,9726,19072.59,699.16,699.16,19072.59,23,1
1254402,2021-12-30,Hale,Alabama,3295,91.00,1065,AL,County,14651,22489.93,621.12,621.12,22489.93,1,2
414426,2021-12-30,Walker,Alabama,12588,388.00,1127,AL,County,63521,19817.07,610.82,610.82,19817.07,15,3
978291,2021-12-30,Crenshaw,Alabama,2712,79.00,1041,AL,County,13772,19692.13,573.63,573.63,19692.13,17,4
867183,2021-12-30,Dallas,Alabama,5487,212.00,1047,AL,County,37196,14751.59,569.95,569.95,14751.59,62,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134461,2021-12-30,Sheridan,Wyoming,6014,61.00,56033,WY,County,30485,19727.73,200.10,200.10,19727.73,11,19
1453326,2021-12-30,Uinta,Wyoming,4154,31.00,56041,WY,County,20226,20537.92,153.27,153.27,20537.92,9,20
1490909,2021-12-30,Lincoln,Wyoming,3087,27.00,56023,WY,County,19830,15567.32,136.16,136.16,15567.32,21,21
975711,2021-12-30,Albany,Wyoming,7048,43.00,56001,WY,County,38880,18127.57,110.60,110.60,18127.57,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,102.30,9766.65,31,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,102.30,9766.65,31,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,102.30,9766.65,31,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,102.30,9766.65,31,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,102.30,9766.65,31,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969775,2021-12-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1969776,2021-12-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1969777,2021-12-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1969778,2021-12-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1249198,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,22489.93,2,1,1.00,nan
1249199,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,22489.93,2,1,0.00,nan
1249200,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,22489.93,2,1,0.00,nan
1249201,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,22489.93,2,1,0.00,nan
1249202,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,621.12,22489.93,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516233,2021-12-26,Crook,Wyoming,1020,22.00,56011,WY,County,7584,13449.37,290.08,290.08,13502.11,11,23,0.00,1.14
1516234,2021-12-27,Crook,Wyoming,1020,22.00,56011,WY,County,7584,13449.37,290.08,290.08,13502.11,11,23,0.00,0.93
1516235,2021-12-28,Crook,Wyoming,1021,22.00,56011,WY,County,7584,13462.55,290.08,290.08,13502.11,11,23,1.00,0.93
1516236,2021-12-29,Crook,Wyoming,1021,22.00,56011,WY,County,7584,13462.55,290.08,290.08,13502.11,11,23,0.00,0.79


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
975659,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,19072.59,1,23,1.00,4.07,0.00,nan
975660,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,19072.59,1,23,0.00,3.93,0.00,nan
975661,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,19072.59,1,23,0.00,3.64,0.00,nan
975662,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,19072.59,1,23,0.00,3.43,0.00,nan
975663,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,19072.59,1,23,0.00,3.14,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409834,2021-12-26,Teton,Wyoming,5598,14.00,56039,WY,County,23464,23857.82,59.67,59.67,25358.00,23,1,0.00,11.50,0.00,0.00
409835,2021-12-27,Teton,Wyoming,5705,14.00,56039,WY,County,23464,24313.84,59.67,59.67,25358.00,23,1,107.00,18.50,0.00,0.00
409836,2021-12-28,Teton,Wyoming,5767,14.00,56039,WY,County,23464,24578.08,59.67,59.67,25358.00,23,1,62.00,22.07,0.00,0.00
409837,2021-12-29,Teton,Wyoming,5835,14.00,56039,WY,County,23464,24867.88,59.67,59.67,25358.00,23,1,68.00,26.79,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-12-30') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
495242,2021-12-30,Imperial,California,41864,799.00,6025,CA,County,181215,23101.84,440.91,440.91,23101.84,1,3,420.00,118.14,0.00,0.79
245636,2021-12-30,San Bernardino,California,395615,6049.00,6071,CA,County,2180085,18146.77,277.47,277.47,18146.77,2,5,1792.00,866.57,2.00,2.57
3532,2021-12-30,Los Angeles,California,1670249,27625.00,6037,CA,County,10039107,16637.43,275.17,275.17,16637.43,3,10,20169.00,8298.50,24.00,16.93
75652,2021-12-30,Shasta,California,26954,491.00,6089,CA,County,180080,14967.79,272.66,272.66,14967.79,4,20,73.00,24.21,0.00,1.07
1197839,2021-12-30,Tuolumne,California,8292,148.00,6109,CA,County,54478,15220.82,271.67,271.67,15220.82,5,17,31.00,17.29,0.00,0.00
870365,2021-12-30,Inyo,California,2743,49.00,6027,CA,County,18039,15205.94,271.63,271.63,15205.94,6,18,29.00,7.21,1.00,0.14
109377,2021-12-30,Stanislaus,California,95133,1473.00,6099,CA,County,550660,17276.18,267.50,267.50,17276.18,7,8,304.00,133.57,5.00,1.00
1056920,2021-12-30,Kings,California,36457,387.00,6031,CA,County,152940,23837.45,253.04,253.04,23837.45,8,2,57.00,37.43,0.00,0.86
788213,2021-12-30,Merced,California,47170,699.00,6047,CA,County,277680,16987.18,251.73,251.73,16987.18,9,9,200.00,60.36,1.00,0.71
135774,2021-12-30,Tulare,California,89497,1160.00,6107,CA,County,466195,19197.33,248.82,248.82,19197.33,10,4,423.00,103.00,4.00,2.86


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1859551,2021-12-30,Lassen,California,8083,61.00,6035,CA,County,30573,26438.36,199.52,199.52,26438.36,18,1,22.00,3.14,0.00,0.00
1056920,2021-12-30,Kings,California,36457,387.00,6031,CA,County,152940,23837.45,253.04,253.04,23837.45,8,2,57.00,37.43,0.00,0.86
495242,2021-12-30,Imperial,California,41864,799.00,6025,CA,County,181215,23101.84,440.91,440.91,23101.84,1,3,420.00,118.14,0.00,0.79
135774,2021-12-30,Tulare,California,89497,1160.00,6107,CA,County,466195,19197.33,248.82,248.82,19197.33,10,4,423.00,103.00,4.00,2.86
245636,2021-12-30,San Bernardino,California,395615,6049.00,6071,CA,County,2180085,18146.77,277.47,277.47,18146.77,2,5,1792.00,866.57,2.00,2.57
309213,2021-12-30,Kern,California,162767,1918.00,6029,CA,County,900202,18081.16,213.06,213.06,18081.16,14,6,0.00,138.71,0.00,2.14
49778,2021-12-30,Madera,California,27268,324.00,6039,CA,County,157327,17332.05,205.94,205.94,17332.05,16,7,90.00,31.64,0.00,0.50
109377,2021-12-30,Stanislaus,California,95133,1473.00,6099,CA,County,550660,17276.18,267.50,267.50,17276.18,7,8,304.00,133.57,5.00,1.00
788213,2021-12-30,Merced,California,47170,699.00,6047,CA,County,277680,16987.18,251.73,251.73,16987.18,9,9,200.00,60.36,1.00,0.71
3532,2021-12-30,Los Angeles,California,1670249,27625.00,6037,CA,County,10039107,16637.43,275.17,275.17,16637.43,3,10,20169.00,8298.50,24.00,16.93


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
495242,2021-12-30,Imperial,California,41864,799.00,6025,CA,County,181215,23101.84,440.91,440.91,23101.84,1,3,420.00,118.14,0.00,0.79
245636,2021-12-30,San Bernardino,California,395615,6049.00,6071,CA,County,2180085,18146.77,277.47,277.47,18146.77,2,5,1792.00,866.57,2.00,2.57
3532,2021-12-30,Los Angeles,California,1670249,27625.00,6037,CA,County,10039107,16637.43,275.17,275.17,16637.43,3,10,20169.00,8298.50,24.00,16.93
75652,2021-12-30,Shasta,California,26954,491.00,6089,CA,County,180080,14967.79,272.66,272.66,14967.79,4,20,73.00,24.21,0.00,1.07
1197839,2021-12-30,Tuolumne,California,8292,148.00,6109,CA,County,54478,15220.82,271.67,271.67,15220.82,5,17,31.00,17.29,0.00,0.00
870365,2021-12-30,Inyo,California,2743,49.00,6027,CA,County,18039,15205.94,271.63,271.63,15205.94,6,18,29.00,7.21,1.00,0.14
109377,2021-12-30,Stanislaus,California,95133,1473.00,6099,CA,County,550660,17276.18,267.50,267.50,17276.18,7,8,304.00,133.57,5.00,1.00
1056920,2021-12-30,Kings,California,36457,387.00,6031,CA,County,152940,23837.45,253.04,253.04,23837.45,8,2,57.00,37.43,0.00,0.86
788213,2021-12-30,Merced,California,47170,699.00,6047,CA,County,277680,16987.18,251.73,251.73,16987.18,9,9,200.00,60.36,1.00,0.71
135774,2021-12-30,Tulare,California,89497,1160.00,6107,CA,County,466195,19197.33,248.82,248.82,19197.33,10,4,423.00,103.00,4.00,2.86


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,32.93
1,03/21/20,Imperial,4,2.21,3,0.00,32.93
2,03/22/20,Imperial,4,2.21,3,0.00,32.93
3,03/23/20,Imperial,4,2.21,3,0.00,26.43
4,03/24/20,Imperial,9,4.97,3,5.00,25.07
...,...,...,...,...,...,...,...
6522,12/26/21,Lassen,8061,26366.40,1,0.00,2.29
6523,12/27/21,Lassen,8061,26366.40,1,0.00,2.29
6524,12/28/21,Lassen,8061,26366.40,1,0.00,1.57
6525,12/29/21,Lassen,8061,26366.40,1,0.00,1.57


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.07
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.07
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.07
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.07
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.07
...,...,...,...,...,...,...,...
6567,12/26/21,Tulare,1137.00,243.89,10,0.00,2.43
6568,12/27/21,Tulare,1137.00,243.89,10,0.00,1.57
6569,12/28/21,Tulare,1146.00,245.82,10,9.00,2.29
6570,12/29/21,Tulare,1156.00,247.96,10,10.00,2.57


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)